In [13]:
# 07_complete_pipeline.py
"""
Complete Research Assistant Pipeline
- Multi-agent architecture (researcher, summarizer, critic, judge)
- Iterative refinement with quality thresholds
- Guardrails at each stage
- Human-in-the-loop approval
"""

import os
import json
import re
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

load_dotenv()

# ============================================================
# CONFIGURATION
# ============================================================

class Config:
    QUALITY_THRESHOLD = 4.0
    JUDGE_CONFIDENCE_THRESHOLD = 0.8
    MAX_ITERATIONS = 3
    MIN_SUMMARY_WORDS = 50
    MAX_SUMMARY_WORDS = 2000
    REQUIRE_HUMAN_APPROVAL = True

# ============================================================
# GUARDRAILS
# ============================================================

class GuardrailError(Exception):
    """Raised when content fails a guardrail check."""
    pass

def validate_word_count(content, min_words=100, max_words=2000):
    """Ensure content is within acceptable length bounds."""
    word_count = len(content.split())
    if word_count < min_words:
        raise GuardrailError(f"Content too short: {word_count} words (minimum: {min_words})")
    if word_count > max_words:
        raise GuardrailError(f"Content too long: {word_count} words (maximum: {max_words})")
    print(f"   ✅ Word count OK: {word_count} words")
    return True

def validate_not_empty(content):
    """Ensure content is not empty or whitespace only."""
    if not content or not content.strip():
        raise GuardrailError("Content is empty")
    print(f"   ✅ Content not empty")
    return True

def validate_no_prohibited_content(content, prohibited_terms=None):
    """Ensure content doesn't contain prohibited terms."""
    if prohibited_terms is None:
        prohibited_terms = ["TODO", "FIXME", "placeholder", "INSERT", "TBD"]
    
    content_lower = content.lower()
    found = [term for term in prohibited_terms if term.lower() in content_lower]
    if found:
        raise GuardrailError(f"Prohibited content detected: {found}")
    print(f"   ✅ No prohibited content found")
    return True

def validate_has_structure(content, required_markers=None):
    """Ensure content has expected structural elements."""
    if required_markers is None:
        required_markers = ["#", "-", "1."]  # Headers, bullets, or numbered lists
    
    has_structure = any(marker in content for marker in required_markers)
    if not has_structure:
        raise GuardrailError(f"Content lacks expected structure (looking for: {required_markers})")
    print(f"   ✅ Content has structure")
    return True

def validate_json_parseable(content):
    """Check if content contains valid JSON (for critic/judge outputs)."""
    # Try to find JSON in the content
    json_match = re.search(r'[\[\{].*[\]\}]', content, re.DOTALL)
    if not json_match:
        raise GuardrailError("No JSON found in content")
    
    try:
        json.loads(json_match.group())
        print(f"   ✅ Valid JSON found")
        return True
    except json.JSONDecodeError as e:
        raise GuardrailError(f"Invalid JSON: {e}")

def validate_critic_scores(content):
    """Validate that critic output contains properly formatted scores."""
    try:
        json_match = re.search(r'\[.*\]', content, re.DOTALL)
        if not json_match:
            raise GuardrailError("Critic output must contain a JSON array")
        
        evaluations = json.loads(json_match.group())
        
        if not evaluations:
            raise GuardrailError("Critic provided no evaluations")
        
        required_fields = ['accuracy', 'clarity', 'completeness']
        for i, eval_item in enumerate(evaluations):
            missing = [f for f in required_fields if f not in eval_item]
            if missing:
                raise GuardrailError(f"Evaluation {i+1} missing fields: {missing}")
            
            for field in required_fields:
                score = eval_item[field]
                if not isinstance(score, (int, float)) or score < 1 or score > 5:
                    raise GuardrailError(f"Invalid score for {field}: {score} (must be 1-5)")
        
        print(f"   ✅ Critic scores valid: {len(evaluations)} evaluations")
        return True
        
    except json.JSONDecodeError as e:
        raise GuardrailError(f"Invalid JSON in critic output: {e}")

def run_guardrails(content, stage="output"):
    """Apply appropriate guardrails based on pipeline stage."""
    
    print(f"\n🛡️  Running guardrails for '{stage}' stage...")
    
    try:
        if stage == "research":
            validate_not_empty(content)
            validate_word_count(content, min_words=30, max_words=5000)
            
        elif stage == "summary":
            validate_not_empty(content)
            validate_word_count(content, min_words=Config.MIN_SUMMARY_WORDS, max_words=Config.MAX_SUMMARY_WORDS)
            validate_has_structure(content)
            
        elif stage == "critique":
            validate_not_empty(content)
            validate_json_parseable(content)
            validate_critic_scores(content)
            
        elif stage == "judge":
            validate_not_empty(content)
            validate_json_parseable(content)
            
        elif stage == "final":
            validate_not_empty(content)
            validate_word_count(content, min_words=100, max_words=3000)
            validate_no_prohibited_content(content)
            validate_has_structure(content)
        
        print(f"✅ All guardrails passed for '{stage}' stage\n")
        return True, None
        
    except GuardrailError as e:
        print(f"❌ Guardrail failed for '{stage}' stage: {e}\n")
        return False, str(e)

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def extract_scores(critique_output):
    """Parse the critic's JSON output to extract average score."""
    try:
        json_match = re.search(r'\[.*\]', critique_output, re.DOTALL)
        if json_match:
            evaluations = json.loads(json_match.group())
            scores = []
            for eval_item in evaluations:
                avg = (eval_item.get('accuracy', 0) + 
                       eval_item.get('clarity', 0) + 
                       eval_item.get('completeness', 0)) / 3
                scores.append(avg)
            return sum(scores) / len(scores) if scores else 0
    except (json.JSONDecodeError, KeyError, TypeError) as e:
        print(f"⚠️  Could not parse critique scores: {e}")
    return 0

def extract_feedback(critique_output):
    """Extract written feedback from critique for next iteration."""
    try:
        json_match = re.search(r'\[.*\]', critique_output, re.DOTALL)
        if json_match:
            evaluations = json.loads(json_match.group())
            feedback_items = [eval_item.get('feedback', '') for eval_item in evaluations]
            if any(feedback_items):
                return "Previous feedback to address:\n- " + "\n- ".join(f for f in feedback_items if f)
    except (json.JSONDecodeError, KeyError, TypeError):
        pass
    return ""

# ============================================================
# AGENTS
# ============================================================

search_tool = SerperDevTool()

researcher = Agent(
    role="Senior Research Analyst",
    goal="Find the most relevant academic papers on a topic",
    backstory="""You are a meticulous researcher who finds high-quality 
    academic sources from top venues like arXiv, NeurIPS, ICML, and ACL.
    You prioritize recent, well-cited papers and provide accurate metadata.""",
    tools=[search_tool],
    verbose=True
)

summarizer = Agent(
    role="Technical Writer",
    goal="Create clear, accurate summaries of research papers",
    backstory="""You are a skilled technical writer who distills complex 
    research into accessible insights. When given feedback, you incorporate 
    it thoughtfully to improve your work. You always use structured formatting
    with headers and bullet points for clarity.""",
    verbose=True
)

critic = Agent(
    role="Quality Reviewer",
    goal="Evaluate summaries for accuracy, clarity, and completeness",
    backstory="""You are a demanding but fair editor with high standards.
    You ALWAYS provide numerical scores (1-5) AND specific actionable feedback.
    You return your evaluations as a valid JSON array.""",
    verbose=True
)

judge = Agent(
    role="Senior Editor",
    goal="Make final publication decisions based on overall quality",
    backstory="""You are the final gatekeeper before publication. You assess 
    whether content meets publication standards holistically. You assign a 
    confidence score (0.0-1.0) and always return your judgment as valid JSON.
    You are conservative: when in doubt, recommend human review.""",
    verbose=True
)

# ============================================================
# PHASE 1: RESEARCH
# ============================================================

def run_research_phase(topic):
    """Phase 1: Research papers on the given topic."""
    
    research_task = Task(
        description=f"""Search for the top 3 most relevant academic papers on: {topic}
        
        For each paper, provide:
        - Title (exact)
        - Authors (first author et al. for papers with many authors)
        - Year and venue
        - Key findings (verbatim abstract excerpt or close paraphrase)
        
        Focus on influential, well-cited papers from reputable venues.
        Format your response with clear headers and structure.
        """,
        expected_output="A structured list of 3 papers with full metadata and findings.",
        agent=researcher
    )
    
    research_crew = Crew(
        agents=[researcher],
        tasks=[research_task],
        process=Process.sequential,
        verbose=True
    )
    
    result = research_crew.kickoff()
    return result.raw

# ============================================================
# PHASE 2: SUMMARIZE + CRITIQUE LOOP
# ============================================================

def run_summarize_critique_loop(research_results):
    """Phase 2: Iteratively summarize and critique until quality threshold is met."""
    
    previous_feedback = ""
    final_summaries = ""
    avg_score = 0
    
    for iteration in range(1, Config.MAX_ITERATIONS + 1):
        print(f"\n📝 Iteration {iteration}/{Config.MAX_ITERATIONS}: Summarizing and critiquing...")
        
        # Build the summarize task with any previous feedback
        feedback_section = f"\n\nIMPORTANT - Address this feedback from the previous iteration:\n{previous_feedback}" if previous_feedback else ""
        
        summarize_task = Task(
            description=f"""Based on the research below, create a summary for each paper.
            
            For each paper, write using this structure:
            
            ## [Paper Title]
            
            **Main Contribution:** 2-3 sentences on what's new/important
            
            **Methodology:** 1-2 sentences on how they did it
            
            **Key Results:** 1-2 sentences on what they found
            
            **Implications:** 1 sentence on why it matters
            
            Write for a technical audience. Be precise but accessible.
            {feedback_section}
            
            RESEARCH TO SUMMARIZE:
            {research_results}
            """,
            expected_output="Three structured paper summaries with contribution, methodology, results, and implications.",
            agent=summarizer
        )
        
        critique_task = Task(
            description="""Review the summaries and evaluate each one.
            
            For each summary, rate on a 1-5 scale:
            - **accuracy**: Does it correctly represent the paper's claims?
            - **clarity**: Is it understandable to the target audience?
            - **completeness**: Are the key points covered?
            
            You MUST return your evaluation as a valid JSON array like this:
```json
            [
                {"paper": "Paper Title", "accuracy": 4, "clarity": 5, "completeness": 4, 
                 "feedback": "Specific suggestions for improvement"},
                {"paper": "Second Paper", "accuracy": 3, "clarity": 4, "completeness": 4,
                 "feedback": "More specific feedback here"}
            ]
```
            
            Scoring guide:
            - 5: Excellent, no improvements needed
            - 4: Good, minor improvements possible
            - 3: Acceptable, some improvements needed
            - 2: Below average, significant improvements needed
            - 1: Poor, major revision required
            
            Always provide specific, actionable feedback for each paper.
            """,
            expected_output="A valid JSON array with scores and feedback for each summary.",
            agent=critic
        )
        
        # Run summarizer + critic
        refine_crew = Crew(
            agents=[summarizer, critic],
            tasks=[summarize_task, critique_task],
            process=Process.sequential,
            verbose=True
        )
        
        result = refine_crew.kickoff()
        final_summaries = result.raw
        
        # Extract scores from the critique portion
        avg_score = extract_scores(final_summaries)
        print(f"\n📊 Iteration {iteration} average score: {avg_score:.2f}")
        
        if avg_score >= Config.QUALITY_THRESHOLD:
            print(f"✅ Quality threshold ({Config.QUALITY_THRESHOLD}) met!")
            break
        
        if iteration < Config.MAX_ITERATIONS:
            previous_feedback = extract_feedback(final_summaries)
            print(f"⚠️  Score {avg_score:.2f} below threshold ({Config.QUALITY_THRESHOLD}). Refining...")
        else:
            print(f"🚨 Max iterations reached. Proceeding with current output.")
    
    return final_summaries, avg_score

# ============================================================
# PHASE 3: JUDGE EVALUATION
# ============================================================

def run_judge_evaluation(summaries):
    """Phase 3: Run the judge to evaluate final summaries."""
    
    judge_task = Task(
        description=f"""Review the final summaries and provide a publication decision.
        
        SUMMARIES TO REVIEW:
        {summaries}
        
        Evaluate the overall quality considering:
        1. Is the information accurate and well-supported?
        2. Is the writing clear and professional?
        3. Does it serve the reader's needs?
        
        You MUST return your judgment as valid JSON in exactly this format:
```json
        {{
            "overall_confidence": 0.85,
            "recommendation": "publish",
            "paper_evaluations": [
                {{"paper": "Title", "confidence": 0.9, "issues": ""}}
            ],
            "general_comments": "Overall assessment here"
        }}
```
        
        Confidence scale:
        - 0.9-1.0: Excellent, publish immediately
        - 0.8-0.9: Good, acceptable for publication
        - 0.7-0.8: Adequate, consider minor revisions
        - Below 0.7: Needs work, recommend human_review
        
        For recommendation, use exactly one of: "publish", "revise", "human_review"
        """,
        expected_output="Valid JSON with overall_confidence, recommendation, paper_evaluations, and general_comments.",
        agent=judge
    )
    
    judge_crew = Crew(
        agents=[judge],
        tasks=[judge_task],
        process=Process.sequential,
        verbose=True
    )
    
    print("\n⚖️  Running Judge evaluation...")
    result = judge_crew.kickoff()
    
    # Parse judge output
    try:
        json_match = re.search(r'\{.*\}', result.raw, re.DOTALL)
        if json_match:
            judgment = json.loads(json_match.group())
            confidence = judgment.get('overall_confidence', 0)
            recommendation = judgment.get('recommendation', 'unknown')
            
            print(f"\n📊 Judge confidence: {confidence:.2%}")
            print(f"📋 Recommendation: {recommendation}")
            
            return confidence, recommendation, judgment, result.raw
    except (json.JSONDecodeError, KeyError) as e:
        print(f"⚠️  Could not parse judge output: {e}")
    
    return 0, "error", {}, result.raw

# ============================================================
# PHASE 4: HUMAN-IN-THE-LOOP
# ============================================================

def get_human_approval(content, context=""):
    """Request human approval for content before proceeding."""
    
    print("\n" + "="*60)
    print("🔔 HUMAN REVIEW REQUESTED")
    print("="*60)
    
    if context:
        print(f"\n📋 Context: {context}")
    
    print("\n" + "-"*40)
    print("CONTENT FOR REVIEW:")
    print("-"*40)
    # Show truncated content if too long
    if len(content) > 3000:
        print(content[:1500])
        print("\n... [content truncated for display] ...\n")
        print(content[-1500:])
    else:
        print(content)
    print("-"*40 + "\n")
    
    while True:
        response = input("Approve this content? [y]es / [n]o / [e]dit with feedback: ").lower().strip()
        
        if response in ['y', 'yes']:
            print("✅ Approved by human reviewer.")
            return True, None
        elif response in ['n', 'no']:
            feedback = input("Please provide feedback for revision: ").strip()
            print("❌ Rejected by human reviewer.")
            return False, feedback
        elif response in ['e', 'edit']:
            feedback = input("Enter your feedback: ").strip()
            print(f"📝 Feedback recorded.")
            return False, feedback
        else:
            print("⚠️  Please enter 'y' (yes), 'n' (no), or 'e' (edit with feedback)")

# ============================================================
# COMPLETE PIPELINE
# ============================================================

def run_complete_pipeline(topic):
    """Execute the complete research assistant pipeline with all components."""
    
    print(f"\n{'='*60}")
    print(f"🔬 COMPLETE RESEARCH ASSISTANT PIPELINE")
    print(f"📚 Topic: {topic}")
    print(f"{'='*60}")
    print(f"\n⚙️  Configuration:")
    print(f"   - Quality threshold: {Config.QUALITY_THRESHOLD}")
    print(f"   - Judge confidence threshold: {Config.JUDGE_CONFIDENCE_THRESHOLD}")
    print(f"   - Max iterations: {Config.MAX_ITERATIONS}")
    print(f"   - Human approval required: {Config.REQUIRE_HUMAN_APPROVAL}")
    
    # ========================================
    # Phase 1: Research
    # ========================================
    print("\n" + "="*50)
    print("📚 PHASE 1: Research")
    print("="*50)
    
    research_results = run_research_phase(topic)
    
    # Guardrail check for research
    passed, error = run_guardrails(research_results, "research")
    if not passed:
        print(f"❌ Pipeline failed at research stage: {error}")
        return None, "failed_research_guardrails"
    
    print("✅ Research phase complete.")
    
    # ========================================
    # Phase 2: Summarize + Critique Loop
    # ========================================
    print("\n" + "="*50)
    print("📝 PHASE 2: Summarization & Critique Loop")
    print("="*50)
    
    summaries, critique_score = run_summarize_critique_loop(research_results)
    
    # Guardrail check for summaries
    passed, error = run_guardrails(summaries, "summary")
    if not passed:
        print(f"⚠️  Summary guardrails failed: {error}")
        print("   Proceeding to judge evaluation anyway...")
    
    print(f"✅ Summarization phase complete. Final critique score: {critique_score:.2f}")
    
    # ========================================
    # Phase 3: Judge Evaluation
    # ========================================
    print("\n" + "="*50)
    print("⚖️  PHASE 3: Judge Evaluation")
    print("="*50)
    
    confidence, recommendation, judgment, judge_raw = run_judge_evaluation(summaries)
    
    # Guardrail check for judge output
    passed, error = run_guardrails(judge_raw, "judge")
    if not passed:
        print(f"⚠️  Judge output guardrails failed: {error}")
        print("   Defaulting to human review...")
        confidence = 0
        recommendation = "human_review"
    
    # ========================================
    # Phase 4: Human Review Decision
    # ========================================
    print("\n" + "="*50)
    print("👤 PHASE 4: Human Review Decision")
    print("="*50)
    
    needs_human_review = (
        Config.REQUIRE_HUMAN_APPROVAL or
        confidence < Config.JUDGE_CONFIDENCE_THRESHOLD or
        recommendation == "human_review"
    )
    
    if needs_human_review:
        reasons = []
        if Config.REQUIRE_HUMAN_APPROVAL:
            reasons.append("human approval required by config")
        if confidence < Config.JUDGE_CONFIDENCE_THRESHOLD:
            reasons.append(f"confidence {confidence:.2%} < threshold {Config.JUDGE_CONFIDENCE_THRESHOLD:.2%}")
        if recommendation == "human_review":
            reasons.append("judge recommended human review")
        
        print(f"\n⚠️  Human review triggered because: {', '.join(reasons)}")
        
        context = f"Judge confidence: {confidence:.2%} | Recommendation: {recommendation}"
        approved, feedback = get_human_approval(summaries, context)
        
        if not approved:
            print(f"\n📝 Human feedback: {feedback}")
            return summaries, "rejected_by_human"
        
        final_status = "approved_by_human"
    else:
        print(f"\n✅ Auto-approval conditions met:")
        print(f"   - Judge confidence: {confidence:.2%} >= {Config.JUDGE_CONFIDENCE_THRESHOLD:.2%}")
        print(f"   - Recommendation: {recommendation}")
        final_status = "auto_approved"
    
    # ========================================
    # Phase 5: Final Guardrails
    # ========================================
    print("\n" + "="*50)
    print("🛡️  PHASE 5: Final Guardrails")
    print("="*50)
    
    passed, error = run_guardrails(summaries, "final")
    if not passed:
        print(f"❌ Final guardrails failed: {error}")
        return summaries, "failed_final_guardrails"
    
    # ========================================
    # Complete!
    # ========================================
    print("\n" + "="*50)
    print("🎉 PIPELINE COMPLETE")
    print("="*50)
    
    return summaries, final_status

# ============================================================
# MAIN
# ============================================================

if __name__ == "__main__":
    # You can change the topic here or make it interactive
    topic = input("Enter research topic (or press Enter for default): ").strip()
    if not topic:
        topic = "large language model alignment"
    
    final_content, status = run_complete_pipeline(topic)
    
    print("\n" + "="*60)
    print(f"🏁 FINAL STATUS: {status.upper()}")
    print("="*60)
    
    if final_content:
        if "approved" in status:
            print("\n📰 PUBLISHED CONTENT:")
            print("-"*40)
            print(final_content)
            print("-"*40)
            print("\n✅ Article published successfully!")
        elif "rejected" in status:
            print("\n📋 DRAFT SAVED FOR REVISION:")
            print("-"*40)
            print(final_content)
            print("-"*40)
            print("\n📝 Please revise based on feedback and re-run.")
        else:
            print("\n⚠️  CONTENT (pipeline did not complete successfully):")
            print("-"*40)
            print(final_content)
            print("-"*40)
    else:
        print("\n❌ No content was produced. Check the error messages above.")

Enter research topic (or press Enter for default):  VLM



🔬 COMPLETE RESEARCH ASSISTANT PIPELINE
📚 Topic: VLM

⚙️  Configuration:
   - Quality threshold: 4.0
   - Judge confidence threshold: 0.8
   - Max iterations: 3
   - Human approval required: True

📚 PHASE 1: Research


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8f0c7ac3-c299-4f47-bea9-069cc8f0c4de                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Search for the top 3 most relevant academic papers on: VLM                                               │
│                                                                                                                 │
│          For each paper, provide:                                                                               │
│          - Title (exact)                                                                                        │
│          - Authors (first author et al. for papers with many authors)                                           │
│          - Year and venue                                                                                       │
│          - Key findings (verbatim abstract excerpt or close paraphrase)                                         │
│                                                                                                                 │
│          Focus on influential, well-cited papers from reputable venues.                                         │
│          Format your response with clear headers and structure.                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bdb8de8c-7025-4495-8382-fdaea254a66b                                                                     │
│  Agent: Senior Editor                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6630e58e-568c-44e4-b64c-8b94f8fa2ac0                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```json                                                                                          │
│  {                                                                                                              │
│      "overall_confidence": 0.83,                                                                                │
│      "recommendation": "revise",                                                                                │
│      "paper_evaluations": [                                                                                     │
│          {                                                                                                      │
│              "paper": "Vision Language Models: A Survey of 26K Papers",                                         │
│              "confidence": 1.0,                                                                                 │
│              "issues": ""                                                                                       │
│          },                                                                                                     │
│          {                                                                                                      │
│              "paper": "A Comprehensive Survey of Vision–Language Models: Pretrained and Domain-Specific         │
│  Approaches",                                                                                                   │
│              "confidence": 0.75,                                                                                │
│              "issues": "Needs clearer explanation of key methodological frameworks."                            │
│          },                                                                                                     │
│          {                                                                                                      │
│              "paper": "EfficientVLM: Fast and Accurate Vision-Language Models via Distillation and Pruning",    │
│              "confidence": 1.0,                                                                                 │
│              "issues": ""                                                                                       │
│          }                                                                                                      │
│      ],                                                                                                         │
│      "general_comments": "The summaries are of good quality overall. However, the second summary requires a     │
│  revision to enhance clarity in its explanation of methodological frameworks. The first and third summaries     │
│  are excellent and fit for publication."                                                                        │
│  }                                                                                                              │
│  ```                                                                                                            │
│                                                                                                                 │
│                                                       


📊 Judge confidence: 83.00%
📋 Recommendation: revise

🛡️  Running guardrails for 'judge' stage...
   ✅ Content not empty
   ✅ Valid JSON found
✅ All guardrails passed for 'judge' stage


👤 PHASE 4: Human Review Decision

⚠️  Human review triggered because: human approval required by config

🔔 HUMAN REVIEW REQUESTED

📋 Context: Judge confidence: 83.00% | Recommendation: revise

----------------------------------------
CONTENT FOR REVIEW:
----------------------------------------
```json
[
    {
        "paper": "Vision Language Models: A Survey of 26K Papers",
        "accuracy": 5,
        "clarity": 5,
        "completeness": 5,
        "feedback": "The summary captures all key aspects of the survey accurately and clearly, presenting the main contributions, methodology, results, and implications in a structured manner."
    },
    {
        "paper": "A Comprehensive Survey of Vision–Language Models: Pretrained and Domain-Specific Approaches",
        "accuracy": 4,
        "clarity": 4

Approve this content? [y]es / [n]o / [e]dit with feedback:  y


✅ Approved by human reviewer.

🛡️  PHASE 5: Final Guardrails

🛡️  Running guardrails for 'final' stage...
   ✅ Content not empty
   ✅ Word count OK: 132 words
   ✅ No prohibited content found
   ✅ Content has structure
✅ All guardrails passed for 'final' stage


🎉 PIPELINE COMPLETE

🏁 FINAL STATUS: APPROVED_BY_HUMAN

📰 PUBLISHED CONTENT:
----------------------------------------
```json
[
    {
        "paper": "Vision Language Models: A Survey of 26K Papers",
        "accuracy": 5,
        "clarity": 5,
        "completeness": 5,
        "feedback": "The summary captures all key aspects of the survey accurately and clearly, presenting the main contributions, methodology, results, and implications in a structured manner."
    },
    {
        "paper": "A Comprehensive Survey of Vision–Language Models: Pretrained and Domain-Specific Approaches",
        "accuracy": 4,
        "clarity": 4,
        "completeness": 4,
        "feedback": "While the summary effectively addresses the main co